<a href="https://colab.research.google.com/github/reddph/Nextdoor-Project/blob/main/nextdoor_electric_sync_v4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Power To Choose - Nextdoor group sync

In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# If you wanted to load csv file from a location in google drive, this would be how to do it, check the next cell,
# I am loading the full csv realtime
#from google.colab import drive
# drive.mount('/content/drive')
#latest_data_in_drive = '/content/drive/MyDrive/ME/nextdoor-app/ptc-rates-06-26-25.csv'


In [2]:
import requests
import io

latest_ptc_url = 'https://www.powertochoose.org/en-us/Plan/ExportToCsv'

# Use requests to get the content from the URL
# By default, requests verifies SSL certificates
try:
    response = requests.get(latest_ptc_url, verify=True) # Added verify=False
    response.raise_for_status() # Raise an HTTPError for bad responses (4xx or 5xx)

    # Read the content into a pandas DataFrame
    df0 = pd.read_csv(io.StringIO(response.text))

    # Set option to display all columns
    pd.set_option('display.max_columns', None)
    #display(df0.head())
    #display(df0.info())

except requests.exceptions.RequestException as e:
    print(f"Error fetching the URL: {e}")
except Exception as e:
    print(f"An error occurred: {e}")

Error fetching the URL: 404 Client Error: Not Found for url: https://www.powertochoose.org/en-us/Plan/ExportToCsv


In [3]:
#first strip out all nan values or empty cells for the [RepCompany] column
df = df0[df0['[RepCompany]'].notna()]
df.shape

NameError: name 'df0' is not defined

In [ ]:
# Need to edit multiple columns that have values of FALSE & TRUE, filter only to include FALSE:
# [PrePaid] [TimeOfUse] [MinUsageFeesCredits]
df = df[df['[PrePaid]'] == False]
df = df[df['[TimeOfUse]'] == False]
df = df[df['[MinUsageFeesCredits]'] == False]
df.shape

In [ ]:
# Filter only to include English for [Language] (last column)
# Filter only include ONCOR for [TduCompanyName] (second column)

df = df[(df['[Language]'] == 'English') & (df['[TduCompanyName]'].str.contains('ONCOR'))]
df.shape

In [ ]:
#Exclude these companies: Tara, Amigos, Just
df_excl = df[(df['[RepCompany]'].str.contains('TARA ENERGY')) | (df['[RepCompany]'].str.contains('AMIGOS ENERGY')) | (df['[RepCompany]'].str.contains('JUST ENERGY')) | (df['[RepCompany]'].str.contains('SPARK ENERGY LLC'))]

#display(df_excl.head(20))

df = df[df['[idKey]'].isin(df_excl['[idKey]']) == False]

#df.head()
df.shape

In [ ]:

#Exclude plans that require Smart Thermostat
df_smartTherm = df[(df['[Product]'].str.contains('Thermostat', case=False, na=False)) | (df['[SpecialTerms]'].str.contains('Thermostat', case=False, na=False))]
#display(df_smartTherm.head(20))

df = df[df['[idKey]'].isin(df_smartTherm['[idKey]']) == False]
df.shape

In [ ]:
# Criteria for excluding plans based on terms in [CancelFee] column in the table
# Had an email discussion with Paul. Paul figured out the following logic for inclusion/exclusion of plans:
# One observation is that about 60% of plans are within a $150 fee range.
# A fee of $100 typically exceeds revenue of 1 month generated by "energy charge" under 8 cents at 1000 kWh.
# Excessive cancellation fees are not in the best interest of the consumer.  Refine as follows:
# If [CancelFee] is numeric and exceeds 150, exclude plan.
# Else if [CancelFee] contains a numeric > 12, exclude plan.

# Parsing pure numeric string to extract the monthly fee for left over months in the contract
# Adding a temporary column CF1 for this scenario
# Regex for pure numeric values is r'([0-9][0-9\.]*$)'
# Examples: 0, 350.00 or 100.00, etc.

pd.set_option('future.no_silent_downcasting', True)

df = df.assign(CF1 = lambda x: (x['[CancelFee]'].str.extract(r'([0-9][0-9\.]*$)')))
df.fillna({'CF1':-1}, inplace=True) # Placing an adhoc large per month fee value for nan on CF1
df[["CF1"]] = df[["CF1"]].astype(float)

# Parsing alphanumeric string to extract the monthly fee for left over months in the contract
# Adding a temporary column CF2 for this scenario
# Regex for pure numeric values is r'([1-9][0-9])[ /a-z]+' which is 2 digits of numeric followed by optional space, slash, followed by text string
# Examples: 20/remaining month or 20 / remaining month, 20 /remaining month, etc.

df = df.assign(CF2 = lambda x: (x['[CancelFee]'].str.extract(r'([1-9][0-9\.]*)[ /a-z]+')))
df.fillna({'CF2':-1}, inplace=True)
df[["CF2"]] = df[["CF2"]].astype(float)

#Exclude plans with CF1 > 150 or CF2 > 12
df_cf_excl = df[(df['CF1'] >= 300) | (df['CF2'] >= 20)]

df = df[df['[idKey]'].isin(df_cf_excl['[idKey]']) == False]

# Sort ascending by [kwh1000]
#df = df.sort_values(by=['[kwh1000]'])

# Drop the temporaty columns CF1 and CF2 created for filtering on CancelFees terms
df.drop(['CF1', 'CF2'], axis=1, inplace=True)

#df.shape
#df_cf_excl.head(40)
#df.head()

In [ ]:
# Cast TermValue values to int as they were converted to decimal on import
df['[TermValue]'] = df['[TermValue]'].astype(int)

In [ ]:
df.shape

In [ ]:
df.rename(columns={'[FactsURL]': 'FactsURL'}, inplace=True)

In [ ]:
factsURL = list(df.loc[:,'FactsURL'])
factsURL

In [ ]:
!pip install python-magic
!pip install pdfplumber
!pip install pypdf
!pip install python-dateutil


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import re
from bs4 import BeautifulSoup
import magic
import pdfplumber
from pypdf import PdfReader
from dateutil.parser import parse
from datetime import datetime

def remove_items(test_list, item):
    # remove the item for all its occurrences
    c = test_list.count(item)
    for i in range(c):
        test_list.remove(item)
    return test_list

def efl_content_extractor(facts_url):
    print(facts_url)

    headers = {'User-Agent': 'Mozilla/5.0'}

    # temporary file that will be overwritten each time
    file_path = '/content/drive/MyDrive/Nextdoor-Project/staging/X_EFL.abc'

    try:
        # Add headers to mimic a browser to prevent access issues
        response = requests.get(facts_url, verify=True, headers = headers )
        response.raise_for_status() # Raise an HTTPError for bad responses (4xx or 5xx)

        with open(file_path, 'wb') as file:
            file.write(response.content)

        # printing the mime type of the file
        file_type = magic.from_file(file_path, mime = True)
        #print('magic: ',file_type)

        if 'html' in file_type:  # html reader

            # Step 1: Parse HTML content
            soup = BeautifulSoup(response.content, 'html.parser')

            # Tried html2text... found it has problems in handling UTF-8 characters in html
            # BeautifulSoup handles much better
            #html_text = html2text.html2text(response.text)

            # Step 2: Extract text
            html_text = soup.get_text()
            return html_text

        elif 'pdf' in file_type:  # pdf reader

            # pymupdf is having troubles with opening the document in some cases.
            # Replacing the read with pypdf package
            # code to extract text from pdf
            # doc = pymupdf.open(file_path)
            # page = doc[0] # load the required page (0-based index)
            # pdf_text = page.get_text() # extract plain text

            # creating a pdf reader object
            #reader = PdfReader(file_path)

            # Open the PDF file
            with pdfplumber.open(file_path) as pdf:
                #if len(pdf.pages) > 0:
                # Access the first page of the PDF
                first_page = pdf.pages[0]

                # Extract the text from the first page
                pdf_text = first_page.extract_text()
                return pdf_text
        else:
            print("Encountered a hitherto unexpected file type", file_type)
            return math.nan
    except requests.exceptions.RequestException as e:
        print("Error fetching the URL: {e}")
    except Exception as e:
        print("An error occurred:", str(e))

In [ ]:
df = df.assign(FactsText = df['FactsURL'].apply(efl_content_extractor))

In [ ]:
df.columns

In [ ]:
df['FactsText'] = df['FactsText'].astype(str)

In [ ]:
import re
import math

def efl_pdf_table_search(facts_url, search_str):
    print(facts_url)
    print(search_str)

    headers = {'User-Agent': 'Mozilla/5.0'}

    # temporary file that will be overwritten each time
    file_path = '/home/reddph/Downloads/temp/X_EFL.xyz'

    try:
        response = requests.get(url,verify=True, headers=headers)
        print(url)

        with open(file_path, 'wb') as file:
            file.write(response.content)

        file_type = magic.from_file(file_path, mime = True)
        #print('magic: ',file_type)

        # Open the PDF file
        pdf = pdfplumber.open(file_path)

        page = pdf.pages[0]

        # Extract tables from the page
        tables = page.extract_tables()

        ec = ['','','']
        # Iterate over each table rows to search for energy charge item
        for i, table in enumerate(tables):
            for row in table:
                if search_str in row:
                    ec = row
                    break

        #print(ec)

        return ec

    except requests.exceptions.RequestException as e:
        print(f"Error fetching the URL: {e}")
    except Exception as e:
        print("An error occurred:", str(e))

In [ ]:
def efl_parse_base_fee(x):
    #print(x)
    m0 = re.search('Average',x)

    if m0:
        buf0 = x[m0.end():]
        m1 = re.search('Base', buf0)
        if m1:
            buf1 = buf0[m1.end():]
            m2 = re.search(r'(\$\s?([0-9]+[\.0-9]*))',buf1)
            if m2:
                val_check = float(m2.group(2))
                if val_check > 0.0 and val_check < 1.0:
                    buf2 = buf1[m2.end():]
                    m3 = re.search(r'\$\s?([0-9]+[\.0-9]*)',buf2)
                    if m3:
                        val_check = float(m3.group(1))
                        if val_check > 0.0 and val_check < 1.0:
                            #print('\n***********BASE CHARGE: 0\n')
                            return 0
                        else:
                            #print('\n***********SECOND SEARCH PASSED...BASE CHARGE:', m3.group(1))
                            return m3.group(1)
                    else:
                        #print('\n*********** SECOND SEARCH FAILED...BASE CHARGE: 0')
                        return 0
                else:
                    #print('***********BASE CHARGE:', m2.group(2))
                    return m2.group(2)
            else:
                #print('*************Base not found')
                return 0
        else:
            #print('***********Key term Base not found, returning 0')
            return 0
    else:
        return math.nan;

# Paul's Specs:
# All EFLs start with the AVERAGE section. That appears to be a standard. The AVERAGE section always have 6 numeric. So skip the first 6 numerics after the word "Average".
# After that, the energy charge will be the first numeric which is shown as cents, such as a cent sign, the word cents, or a value less than 1, greater than zero.
# As mentioned before, when expressed as cents (>1), the range is 3.0 to 25.0 cents

def efl_parse_energy_charge(x):
    #print(x)
    clean_txt = re.sub(r",","",x)
    # Replace embedded whitespaces in decimal strings
    # example 14. 2
    clean_txt = re.sub(r'(\d{2}\.) (\d)',r'\1\2', clean_txt)

    #print(clean_txt)

    m0 = re.search('Average',clean_txt)
    if m0:
        buf1 = clean_txt[m0.end():]
        regex_ec = re.compile(r'((\$|\$ |\$  )?([\.0-9\-]+)(cents| cents|¢| ¢)?)')
        regex_pref = re.compile(r'\$')
        regex_suff = re.compile(r'cent|¢')
        numList = re.findall(regex_ec,buf1)
        numList = remove_items(numList,('.','','.',''))
        numList = remove_items(numList,('. ', '', '',''))
        numList = remove_items(numList,('. ', '.', '', ''))
        numList = remove_items(numList,('. ', '. ', '', ''))
        numList = remove_items(numList,('-', '', '-', ''))
        cleanList = numList
        #print(cleanList)
        ec_df = pd.DataFrame(cleanList, columns=['Expr', 'Prefix', 'Item','Suffix'])
        #print(ec_df.iloc[:,[1,2,3]])
        for i in range(len(cleanList)):
            if i < 6:
                continue
            else:
                if i >= 6:
                    sitem = ec_df.iloc[i,2]
                    sprefix = ec_df.iloc[i,1]
                    ssuffix = ec_df.iloc[i,3]
                    try:
                        item = float(sitem)
                        if ((item >= 3.0 and item <= 25.0 and (regex_suff.search(ssuffix) != None)) or (item > 0 and item < 1 and (regex_pref.search(sprefix) != None))):
                            #print('\n*************selected item:',item)
                            return item
                        else:
                            continue
                    except ValueError:
                        continue
        return math.nan
    else:
        #print('\n**********No match found for Average')
        return math.nan


In [ ]:
def efl_parse_date_from_efl(efl_text):

    patterns = [
        r'(\d{4})-(\d{2})-(\d{2})',  # YYYY-MM-DD
        r'(\d{1,2})/(\d{1,2})/(\d{4})',  # MM/DD/YYYY
        r'(\d{2})-([A-Za-z]{3})-(\d{4})',  # DD-Mon-YYYY
        r'([A-Za-z]+) (\d{1,2},) (\d{4})',  # Month day, Year
        r'(\d{2})-(\d{2})-(\d{4})',  # MM-DD-YYYY
        r'(\d)/(\d{1,2})/(\d{2,4})'  # M/DD/YY
    ]

    dt_str = datetime.strptime('1900-01-01',"%Y-%m-%d")

    clean_txt = re.sub(r'(\d{4}-\d) (\d-\d{2})',r'\1\2',efl_text)

    for pattern in patterns:
        #print(pattern)
        matches = re.finditer(pattern,clean_txt)

        for match in matches:
            mgroup = match.group()
            try:
                if pattern == patterns[0]:
                    dt_str = max(dt_str,datetime.strptime(mgroup, "%Y-%m-%d"))
                elif pattern == patterns[1]:
                    dt_str = max(dt_str,datetime.strptime(mgroup, "%m/%d/%Y"))
                elif pattern == patterns[2]:
                    dt_str = max(dt_str,datetime.strptime(mgroup, "%d-%b-%Y"))
                elif pattern == patterns[3]:
                    dt_str = max(dt_str,datetime.strptime(mgroup, "%B %d, %Y"))
                elif pattern == patterns[4]:
                    dt_str = max(dt_str,datetime.strptime(mgroup, "%m-%d-%Y"))
                else:
                    dt_str = max(dt_str,datetime.strptime(mgroup, "%m/%d/%Y"))
            except ValueError:
                print("error in date string")

    #if (dt_str == datetime.strptime('1900-01-01',"%Y-%m-%d")):
    #    print(efl_text)

    return(dt_str.strftime("%Y-%m-%d"))


In [ ]:
df = df.assign(BaseFee = df['FactsText'].apply(efl_parse_base_fee))

In [ ]:
df = df.assign(EnergyCharge = df['FactsText'].apply(efl_parse_energy_charge))

In [ ]:
df = df.assign(EffectiveDate = df['FactsText'].apply(efl_parse_date_from_efl))

In [ ]:
df.loc[:,['[RepCompany]','[TermValue]','[CancelFee]','BaseFee','EnergyCharge','EffectiveDate','FactsURL']]

In [ ]:
df.drop('FactsText', axis=1, inplace=True)

In [ ]:
# Rename the computed columns:
df.rename(columns={"BaseFee": "BASE_FEE", "EnergyCharge": "ENERGY_CHARGE", "EffectiveDate":"PUBLISHED_DATE"}, inplace=True)

In [ ]:
df.columns

In [ ]:
df[['ENERGY_CHARGE']] = df[['ENERGY_CHARGE']].astype(float)

In [ ]:
#  If EC < 1 cent, EC=EC*100   (Case: SoFed & RANCHERO )
df['ENERGY_CHARGE'] = np.where(df['ENERGY_CHARGE'] < 1.0, df['ENERGY_CHARGE'] * 100, df['ENERGY_CHARGE'])

# For all EC, display with 3 decimal places (N.NNN)
#df['ENERGY_CHARGE'] = df['ENERGY_CHARGE'].round(3)

In [ ]:
# Adjusted EC (@1000 kwh) = EC + base/1000
df = df.assign(Adjusted_EC = df['ENERGY_CHARGE'] + df['BASE_FEE']/1000.0)

In [ ]:
df.loc[:,['[RepCompany]','[TermValue]','[CancelFee]','BASE_FEE','ENERGY_CHARGE','Adjusted_EC','PUBLISHED_DATE']]

In [ ]:
# Create separate groupings by term values: 12-month, 3-month, 30-day (variable rate), other
df_12m = df[df['[TermValue]']==12].sort_values(by='Adjusted_EC',ascending=True).head(7)
df_3m = df[df['[TermValue]']==3].sort_values(by='Adjusted_EC',ascending=True).head(7)
df_4m = df[df['[TermValue]']==4].sort_values(by='Adjusted_EC',ascending=True).head(7)
df_1m = df[df['[TermValue]']==1].sort_values(by='Adjusted_EC',ascending=True).head(7)
# Group all other terms into other dataframe
excl_terms = [1,3,4,12]
df_other = df[~df['[TermValue]'].isin(excl_terms)].sort_values(by='Adjusted_EC',ascending=True).head(7)

In [ ]:
df_1m

In [ ]:
!rm /etc/localtime
!ln -s /usr/share/zoneinfo/US/Central /etc/localtime
!date

In [ ]:
from datetime import datetime
import pytz
local_tz = pytz.timezone('America/Chicago')

In [ ]:
def append_timestamp_to_name(prefix,suffix):
    current_datetime = datetime.now(local_tz).strftime("%Y-%m-%d_%H-%M-%S")

    # convert datetime obj to string
    str_current_datetime = str(current_datetime)

    return prefix + "__" + str_current_datetime + '.' + suffix

def generate_filename_by_term(folder,fname,term):
    return folder + '/' + fname + '_' + term

In [ ]:
#df_12m['ENERGY_CHARGE'] = df_12m['ENERGY_CHARGE'].astype(str).map('{:.3f}'.format
myfolder = '/content/drive/MyDrive/Nextdoor-Project/export'
myprefix = 'PriceSheet'

myterm = '12m'
mypath = append_timestamp_to_name(generate_filename_by_term(myfolder,myprefix,myterm),'csv')
df_12m.to_csv(mypath, sep='\t', index=False, encoding='utf-8')

myterm = '3m'
mypath = append_timestamp_to_name(generate_filename_by_term(myfolder,myprefix,myterm),'csv')
df_3m.to_csv(mypath, sep='\t', index=False, encoding='utf-8')

myterm = '4m'
mypath = append_timestamp_to_name(generate_filename_by_term(myfolder,myprefix,myterm),'csv')
df_4m.to_csv(mypath, sep='\t', index=False, encoding='utf-8')

myterm = '1m'
mypath = append_timestamp_to_name(generate_filename_by_term(myfolder,myprefix,myterm),'csv')
df_1m.to_csv(mypath, sep='\t',index=False, encoding='utf-8')

myterm = 'other'
mypath = append_timestamp_to_name(generate_filename_by_term(myfolder,myprefix,myterm),'csv')
df_other.to_csv(mypath, sep='\t',index=False, encoding='utf-8')

In [ ]:
df_other

In [ ]:
df